In [1]:
import pandas as pd
import scipy as sc
import numpy as np
import sklearn
import pickle
import pathlib as Path
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sb
sb.set(font='IPAexGothic')

import multiprocessing
import itertools
import collections
import datetime
import gc

from tqdm._tqdm_notebook import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 50)

data_path = Path.Path("../data")
result_path = Path.Path("../result")

prefix = 'ana002'

# データ抽出
> 訓練データとテストデータ

In [19]:
# 訓練データ
train_df = pd.read_pickle(data_path / 'train_df.pkl').sort_index()
# テストデータ
test_df = pd.read_pickle(data_path / 'test_df.pkl').sort_index()

# データ結合
> 処理を一括化したいので、訓練データとテストデータを縦結合

In [20]:
# 訓練データ
# データの識別フラグを追加
train_df['flg_train'] = 1

In [21]:
train_df

,totalPitchingCount,B,S,O,b1,b2,b3,pitcher,pitcherHand,batter,batterHand,gameID,inning,pitchType,speed,ballPositionLabel,ballX,ballY,dir,dist,battingType,isOuts,y,flg_train
id,,,,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,0,False,False,False,今永 昇太,L,ピレラ,R,20202173,1回表,ストレート,149km/h,内角低め,17,J,NaN,NaN,NaN,NaN,0,1
1,2,1,0,0,False,False,False,今永 昇太,L,ピレラ,R,20202173,1回表,ストレート,149km/h,内角低め,14,I,NaN,NaN,NaN,NaN,1,1
2,3,1,1,0,False,False,False,今永 昇太,L,ピレラ,R,20202173,1回表,チェンジアップ,137km/h,外角高め,8,D,NaN,NaN,NaN,NaN,0,1
3,4,2,1,0,False,False,False,今永 昇太,L,ピレラ,R,20202173,1回表,スライダー,138km/h,内角中心,21,G,NaN,NaN,NaN,NaN,2,1
4,5,2,2,0,False,False,False,今永 昇太,L,ピレラ,R,20202173,1回表,チェンジアップ,136km/h,外角中心,7,F,S,38.3,G,False,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20395,2,1,0,2,False,False,False,森 唯斗,R,大田 泰示,R,20202118,9回裏,カットファストボール,143km/h,外角中心,7,F,NaN,NaN,NaN,NaN,2,1
20396,3,1,1,2,False,False,False,森 唯斗,R,大田 泰示,R,20202118,9回裏,カーブ,120km/h,真ん中低め,12,K,NaN,NaN,NaN,NaN,0,1
20397,4,2,1,2,False,False,False,森 唯斗,R,大田 泰示,R,20202118,9回裏,カーブ,120km/h,真ん中低め,10,H,NaN,NaN,NaN,NaN,1,1


In [22]:
# テストデータ
test_df['flg_train'] = 0
# 目的変数はnp.nanを入れておく
test_df['y'] = np.nan

test_df

,totalPitchingCount,B,S,O,b1,b2,b3,pitcher,pitcherHand,batter,batterHand,gameID,inning,flg_train,y
id,,,,,,,,,,,,,,,
0,2,1,0,0,False,False,False,遠藤 淳志,R,乙坂 智,L,20202564,2回表,0,NaN
1,1,0,0,0,False,False,False,バンデンハーク,R,西川 遥輝,L,20202106,3回裏,0,NaN
2,7,3,2,2,True,False,False,スアレス,R,堂林 翔太,R,20203305,9回裏,0,NaN
3,1,0,0,2,True,False,False,クック,R,井領 雅貴,L,20202650,3回裏,0,NaN
4,2,0,0,2,False,False,False,則本 昂大,R,安達 了一,R,20202339,2回表,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33803,2,0,1,0,False,False,False,床田 寛樹,L,坂口 智隆,L,20202023,5回表,0,NaN
33804,1,0,0,0,False,False,False,堀岡 隼人,R,メヒア,R,20202640,9回表,0,NaN
33805,1,0,0,0,True,False,False,ディプラン,R,鈴木 誠也,R,20202864,7回裏,0,NaN


In [23]:
# データを縦結合
all_df = pd.concat(
    
            [
                # 訓練データ
                train_df, 
                # テストデータ
                test_df
            ],
            # 縦結合
            axis=0
        )

C:\Users\livin\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [24]:
all_df

,B,O,S,b1,b2,b3,ballPositionLabel,ballX,ballY,batter,batterHand,battingType,dir,dist,flg_train,gameID,inning,isOuts,pitchType,pitcher,pitcherHand,speed,totalPitchingCount,y
id,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,False,False,False,内角低め,17.0,J,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,149km/h,1,0.0
1,1,0,0,False,False,False,内角低め,14.0,I,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,149km/h,2,1.0
2,1,0,1,False,False,False,外角高め,8.0,D,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,チェンジアップ,今永 昇太,L,137km/h,3,0.0
3,2,0,1,False,False,False,内角中心,21.0,G,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,スライダー,今永 昇太,L,138km/h,4,2.0
4,2,0,2,False,False,False,外角中心,7.0,F,ピレラ,R,G,S,38.3,1,20202173,1回表,False,チェンジアップ,今永 昇太,L,136km/h,5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33803,0,0,1,False,False,False,NaN,NaN,NaN,坂口 智隆,L,NaN,NaN,NaN,0,20202023,5回表,NaN,NaN,床田 寛樹,L,NaN,2,NaN
33804,0,0,0,False,False,False,NaN,NaN,NaN,メヒア,R,NaN,NaN,NaN,0,20202640,9回表,NaN,NaN,堀岡 隼人,R,NaN,1,NaN
33805,0,0,0,True,False,False,NaN,NaN,NaN,鈴木 誠也,R,NaN,NaN,NaN,0,20202864,7回裏,NaN,NaN,ディプラン,R,NaN,1,NaN


# データの保存
> pickle形式

In [25]:
all_df.to_pickle(data_path / 'all_df.pkl')